In [75]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

#read images
images = glob("../imgs/*")
print(f"Images: {len(images)}")

if not os.path.exists("results/"):
    os.makedirs("results")

#separate images by name and store in names dictionary
dic = dict()
for x in tqdm(images, total=len(images)):
        """ Extracting the name """
        name = x.split("\\")[-1] #windows format
        nameVec = []
        nameVec.append(x)
        nameKey = name.split("_")[2]
        print(nameKey)
        if nameKey not in dic:
            dic[nameKey] = []
        dic[nameKey].append(nameVec[0])

#for each key read images and put in images dic
image = dict()
for key in dic:
      image[key] = []
      for value in dic[key]:    
          image[key].append(cv2.imread(value, cv2.IMREAD_COLOR))
print("image dic = " , image)

#for each key merge images inside and save in path

        
imagelist = [len(image[key])]
i = 0
for key in image:
    print("i = ", i)
    imagelist[i] = []
    for j in range(len(image[key])):
        print("j = ", j)
        if j == 0:
            imagelist[i] = image[key][j]
        else:
            imagelist[i] = imagelist[i]*image[key][j]
    name = "".join([key,".jpg"]) 
    print(name)
    save_image_path = os.path.join("results", name)
    cv2.imwrite(save_image_path, imagelist[i])
    i += 1


Images: 3


100%|██████████| 3/3 [00:00<?, ?it/s]

00a56935
00a56935
00a56935
image dic =  {'00a56935': [array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1080 is different from 3)